In [18]:
from sklearn.datasets import make_regression
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [19]:
X, y = make_regression(n_samples=600, n_features=5, n_informative=8, noise=15, random_state=41)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4', 'f5'])
y = pd.Series(y, name='target')

In [20]:
X

,f1,f2,f3,f4,f5
0,-0.986257,1.745565,-1.200874,-0.047218,0.267876
1,2.015698,0.488657,0.996674,0.653400,0.065277
2,1.701293,-1.477120,2.039221,-0.025737,0.205535
3,-0.301803,-1.172736,0.528437,0.770789,0.158079
4,1.176931,0.894768,-0.556505,-0.974740,-1.182949
...,...,...,...,...,...
595,-1.451426,-0.050116,-0.249363,-0.384609,0.653114
596,-0.482776,-0.501015,0.504170,-1.131012,0.970515
597,1.880913,-0.866513,0.057113,-0.284894,1.084699
598,1.116665,-0.965840,-1.428672,0.140047,1.800045


In [21]:
y

0      -37.051235
1      355.137614
2      210.941483
3      -38.814320
4       99.142999
          ...    
595   -195.632726
596    -99.735042
597    112.311375
598    -73.126157
599    148.894350
Name: target, Length: 600, dtype: float64

In [22]:
np.random.seed(81)
np.random.rand(X.shape[1])

array([0.35541912, 0.34084888, 0.93310644, 0.91381919, 0.83746473])

# В этой части нужно реализовать подсчет метрики  
- mae
- mse
- rmse
- mape
- r2
  
По умолчанию: None


###### на метриках мы будем оценить качество модели

In [25]:
a = np.array([[3, 2], 
             [2, 1]])
b = np.array([[1, 3, 8], 
             [3, 2, 4]])

In [26]:
a = pd.DataFrame(a)
b = pd.DataFrame(b)

In [27]:
type(a.dot(b))

pandas.core.frame.DataFrame

In [28]:
np.zeros((2, 4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [29]:
class MyLineReg():
    def __init__(self, n_iter=100, learning_rate=0.1, metric=None, weights=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.weights = weights
        self.best_score = None
        
    def __str__(self):
        return f'MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'

    # Напишем метод с приватным модификатором доступа для подсчета разнных метрик 
    def __calculate_metric(self, pred: pd.DataFrame, y_true: pd.Series, metric:'str'):
        """
        Вычисляет значение метрики.

        Параметры:
        pred (np.array): Предсказанные значения.
        y_true (np.array): Фактические значения.
        metric (str): Название метрики.

        Возвращает:
        float: Значение метрики.
        """
        if metric == 'mae':
            return np.mean(np.abs(pred - y_true))
        elif metric == 'mse':
            return np.mean((pred - y_true)**2)
        elif metric == 'rmse':
            return np.sqrt(np.mean((pred - y_true)**2))
        elif metric == 'mape':
            return 100 * np.mean(np.abs((pred - y_true) / y_true))
        elif metric == 'r2':
            ss_residual = np.sum((pred - y_true)**2)
            ss_total = np.sum((y_true - np.mean(y_true))**2)
            return 1 - (ss_residual / ss_total)
        else:
            raise ValueError(f"Unknown metric: {metric}")
        
    def fit(self, X: pd.DataFrame, y: pd.Series, verbose: bool = False):
        """
        Обучение модели линейной регрессии.

        Параметры:
        X (pd.DataFrame): Матрица признаков.
        y (pd.Series): Вектор целевых значений.
        verbose (bool): Флаг для вывода информации о процессе обучения.
        """
        X_copy = X.copy()               # создадим копию нашей матрицы фичей, что бы не изменить изначальный
        X_copy.insert(0, 'base', 1)     # допишем слева столбик из 1 для свободного члела
        w = np.ones(X_copy.shape[1])    # составим вектор весов, заполненный из 1

        if verbose:                   # Напишем подсчет ошибки до начала обучения
            initial_MSE = np.mean((np.dot(X_copy, w) - y)**2)
            if self.metric is None:                          # Если не задана метрика, то в логи выводим только MSE
                print(f'start | loss: {initial_MSE:0.2f}')   
            else:                                            # Если задана, то выводим ошибку на MSE и на выбранной нами метрике
                print(f'start | loss: {initial_MSE:0.2f} | {self.metric}: {self.__calculate_metric(pred=X_copy.dot(w), y_true=y, metric=self.metric):0.2f}')
            
        for i in range(self.n_iter): # Напишем цикл обучения
            pred = np.dot(X_copy, w) # cчитаем предасказания модели 
            error = pred - y         # вычисляем ошибку (предсказания - реальные значения)
            grad = (2/len(X_copy)) * np.dot(error, X_copy) # вычисляем градиенты по весам
            #w -= self.learning_rate * grad  # обновляем веса 

            if self.metric:         # Если задана метрика, то считаем его и обновляем self.best_score
                metric_loss = self.__calculate_metric(pred = X_copy.dot(w), y_true=y, metric=self.metric) # Считаем выбранную метрику
                self.best_score = metric_loss

            if verbose:     # выводим логи 
                loss = np.mean(error**2)
                if self.metric is None and i % 10 == 0:        # Eсли метрика не указана, то в логи выыодим итерацию и ошибку на MSE
                    print(f'Iteration {i}| loss: {loss}')
                elif self.metric and (i % 10 == 0 or self.n_iter - 1 == i): # Если указана метрика, то выводим ошибку MSE и на метрике
                    print(f'Iteration {i}| loss: {loss:0.2f} | {self.metric}: {metric_loss:0.2f}')
                    
            w -= self.learning_rate * grad  # обновляем веса, обновляем тут, чтобы ошибка считалась правильно       
        # сохроняем веса
        self.weights = w 
    def predict(self, X: pd.DataFrame)->np.array:
        """
        Предсказание целевых значений.

        Параметры:
        X (pd.DataFrame): Матрица признаков.

        Возвращает:
        np.array: Предсказанные значения.
        """
        X_copy = X.copy()            # делаем копию матрицы фичей, чтобы не менять изначальный датафрейм
        X_copy.insert(0, 'base', 1)  # допишем слева столбик из 1 для свободного члена
        predict = X_copy.dot(self.weights) # делаем предсказания X_copy @ self.weights
        return predict 

    def get_best_score(self):  # Метод для вывода лучшего качетва на выбранной метрике
        if self.metric is None:
            raise ValueError("Metric was not set during model initialization.")
        return self.best_score

    def get_coef(self):
        return self.weights[1:] # метод для вывода весов начиная с 1го заначения            


In [64]:
model = MyLineReg(metric='mse')

In [66]:
model.fit(X, y, verbose=True)

start | loss: 33311.28 | mse: 33311.28
Iteration 0| loss: 33311.28 | mse: 33311.28
Iteration 10| loss: 509.73 | mse: 509.73
Iteration 20| loss: 217.30 | mse: 217.30
Iteration 30| loss: 214.38 | mse: 214.38
Iteration 40| loss: 214.35 | mse: 214.35
Iteration 50| loss: 214.35 | mse: 214.35
Iteration 60| loss: 214.35 | mse: 214.35
Iteration 70| loss: 214.35 | mse: 214.35
Iteration 80| loss: 214.35 | mse: 214.35
Iteration 90| loss: 214.35 | mse: 214.35
Iteration 99| loss: 214.35 | mse: 214.35


In [68]:
model.predict(X)

0      -41.031372
1      378.326403
2      220.320179
3      -56.183340
4       86.931266
          ...    
595   -185.719279
596    -94.512972
597    103.629790
598    -96.936575
599    147.989363
Length: 600, dtype: float64

In [70]:
y

0      -37.051235
1      355.137614
2      210.941483
3      -38.814320
4       99.142999
          ...    
595   -195.632726
596    -99.735042
597    112.311375
598    -73.126157
599    148.894350
Name: target, Length: 600, dtype: float64

In [72]:
model.weights

array([-1.27758975, 99.20498043, 99.87428665, 97.62035544, 50.07913291,
       12.48797475])

In [74]:
model.get_coef()

array([99.20498043, 99.87428665, 97.62035544, 50.07913291, 12.48797475])